## Setup directories

In [ ]:
# input files
kgtk_wn_file="../tmp/kgtk_wordnet.tsv"
kgtk_cn_file="../tmp/kgtk_conceptnet.tsv"

# output files
wn_gold_file="wn_gold_all.tsv"
wn_gold_200_file="wn_gold_200.tsv"
wn_mrs_prediction_file="wn_MRS_200.tsv"
wn_mfs_prediction_file="wn_MFS_200.tsv"
wn_stb_prediction_file="wn_STB_200.tsv"
wn_str_prediction_file="wn_STR_200.tsv"
cn_test_1k_file="cn_test_1k.tsv"
cn_prediction_file="cn_predict_1k.tsv"

## Generate wn_gold_all.tsv file

In [ ]:
import rltk
import csv
import matplotlib.pyplot as plt
import random
from nltk.corpus import wordnet as wn
import re
from itertools import combinations
from sentence_transformers import SentenceTransformer

In [ ]:
def load_file(filename):
    with open(filename, "r",encoding="UTF-8") as f:
        """
        load data
        token is split by "\t"
        """
        head = f.readline().strip().split("\t")
        lines = []

        for line in f:
            lines.append(line.strip().split("\t"))
    return head, lines
head, lines = load_file(kgtk_wn_file)

In [ ]:
# what does the head look like
head

In [ ]:
def multiple_labels(node_labels,node_noid):
    """
    Obtain label whose leven distance is most close to the node id
    """
    node_res=[float("inf"),""]
    for label in node_labels.split("|"):
        # remove ""
        label = label.replace('"',"").replace("\\'","'")
        dis = rltk.levenshtein_distance(node_noid, label)
        temp = [dis,label]
        
        if temp < node_res:
            node_res  = temp
            
    return node_res[1]

In [ ]:
def generate_gold_file(lines):
    # Transfer line to head 'node1;label','relation','node2;label','node1','node2'
    wn_gold_all = []
    i = 0
    for line in lines:
        #change column to node1;label, relation, node2;label, node1, node2
        node1_id = line[0]
        relation = line[1]
        node2_id = line[2]
        node1_labels = line[3]
        node2_labels = line[4]
        # modeify the node labels, check with leve distance
        
        #node1_noid = node1.split(":")[1].split(".")[0]
        #node2_noid = node2.split(":")[1].split(".")[0]

        node1_label = multiple_labels(node1_labels,node1_id)
        node2_label = multiple_labels(node2_labels,node2_id)

        wn_gold_all.append([node1_label, relation, node2_label, node1_id, node2_id])
        print(f"\r {i}/{len(lines)}", end="")
        i += 1
        
    return wn_gold_all

In [ ]:
wn_gold_all = generate_gold_file(lines)
# example of new dataset
wn_gold_all[0]

In [ ]:
# new head
new_head = ['node1;label','relation','node2;label','node1','node2']
# write in to file
with open(wn_gold_file, "w", newline='') as f:
    writer = csv.writer(f, delimiter='\t')
    writer.writerow(new_head)
    writer.writerows(wn_gold_all)

In [ ]:
wn_gold_all[:10]

## statistics calculation

In [ ]:
def distribution(wn_gold_all):
    # obtain the dsitribution of each label
    # output: {label1-> str: num->integer}
    distri = dict()
    for line in wn_gold_all:
        node1_label = line[0]
        node2_label = line[2]
        node1_id = line[3]
        node2_id = line[4]
        temp1 = distri.get(node1_label,set())
        temp1.add(node1_id)
        temp2 = distri.get(node2_label,set())
        temp2.add(node2_id)
        distri[node1_label] = temp1
        distri[node2_label] = temp2
    
    for item in distri:
        distri[item] = len(distri[item])
    return distri

In [ ]:
distri = distribution(wn_gold_all)
plt.hist(distri.values(),log=True,bins=50)
print("mean ambiguity of label:", sum(distri.values())/len(distri), "size of records:", len(wn_gold_all), "num of distinct labels:", len(distri))

In [ ]:
for item in distri:
    if distri[item] > 20:
        print(item)

## Random Pick

In [ ]:
# random pick 200 records
wn_gold_200 = random.choices(wn_gold_all, k=200)

In [ ]:
wn_gold_200[:3]

In [ ]:
new_head = ['node1;label','relation','node2;label','node1','node2']
# write in to file 200 records
with open(wn_gold_200_file, "w", newline='') as f:
    writer = csv.writer(f, delimiter='\t')
    writer.writerow(new_head)
    writer.writerows(wn_gold_200)

## Get synsets

In [ ]:
all_synsets = list(wn.all_synsets())

In [ ]:
# obtain the regex structure of word combination
regex_dis = dict()
for synset in all_synsets:
    name = synset.name().split(".")[0]
    regex_temple = ""
    for w in name:
        if not regex_temple:
            if w.isalpha():
                regex_temple += "x"
            else:
                regex_temple += w
                
            continue
        
        if w.isalpha() and (not regex_temple[-1].isalpha()):
            regex_temple += "x"
            
        elif not w.isalpha():
            regex_temple += w
    #if regex_temple == "x_x_x_x'x_x":
    #   print(name)
    regex_dis[regex_temple] = name

In [ ]:
regex_dis

In [ ]:
def place_ones(size, count):
    for positions in combinations(range(size), count):
        p = [0] * size

        for i in positions:
            p[i] = 1

        yield p
        
# permutations of list n without repitation
def permu(n):
    comb = []
    for i in range(n+1):
        comb += place_ones(n,i)
        
    return comb

def replace_str(string, replace_w, idx):
    # replace whitespace to _ or -
    return string[:idx] + replace_w +string[idx+1:]

In [ ]:
def transfer_words(label):
    # accoording to the labels, generate label that can be recognized by wn interface from NLTK
    idx_list = [x for x, v in enumerate(label) if v == ' ']
    
    if not idx_list:
        #no whitespace in words
        yield label
    else:
        # whitespace in words
        combs = permu(len(idx_list))
        for comb in combs:
            for idx, status, in zip(idx_list, comb):
                if status:
                    label = replace_str(label, "-", idx)
                else:
                    label = replace_str(label, "_", idx)
            yield label
            
def generate_synsets(labels):
    # According to the generation of labels, obtain the synsets
    for label in labels:
        synsets = list(wn.synsets(label))
        
        if synsets:
            return synsets, label
        
    return [], label

def generate_candidates(label):
    candidates,_ = generate_synsets(transfer_words(label))
    return candidates

In [ ]:
# example of word combination
generate_candidates("far cry")

In [ ]:
# example of word combination
generate_candidates("ladder back")

In [ ]:
wn.synsets("far cry"), wn.synsets("ladder back"),wn.synsets("far_cry"),wn.synsets("ladder-back")

## Random Baseline

In [ ]:
def MRS(wn_gold):
    # Random Baseline calculation
    wn_predict = []
    for line in wn_gold:
        label1  = line[0]
        label2 = line[2]
        relationship = line[1]
        
        candidates1 = generate_candidates(label1)
        candidates2 = generate_candidates(label2)
        
        if candidates1:
            node1_id = random.choice(candidates1)
        else:
            print(label1)
            #_ = label1
            node1_id = ""
        
        if candidates2:
            node2_id = random.choice(candidates2)
        else:
            print(label2)
            #_ = label2
            node2_id = ""
        
        #print(node2_id)
        wn_predict.append([label1, relationship, label2, node1_id, node2_id])
        
    return wn_predict

In [ ]:
wn_predict_200 = MRS(wn_gold_200)
wn_predict_200[:10]

In [ ]:
def synset2str(syn):
    # transfer synset to str in gold file
    # print(syn), print(type(syn))
    if type(syn) == str:
        return "wn:"
    return "wn:"+ syn.name()

def validation(wn_predict,wn_gold):
    # check accracy of prediction
    # accuracy1: if label is correct, true positive +1
    # accuracy2: iff two labels in one record are correct (record is correct), true positive +1
    correct1 = 0
    correct2 = 0
    for predict, actual in zip(wn_predict, wn_gold):
        #print(predict, actual)
        judge = [synset2str(predict[3]) == actual[3],synset2str(predict[4]) == actual[4]]
        #print(predict[3],actual[3])
        if judge[0]:
            correct1 += 1
            
        if judge[1]:
            correct1 += 1
            
        if all(judge):
            correct2 += 1
            
    return correct1/(len(wn_predict)*2), correct2/len(wn_predict)

def write_prediction(filename, lines):
    # write prediction to file
    new_head = ['node1;label','relation','node2;label','node1','node2']
    # write in to file 200 records
    with open(filename, "w", newline='',encoding="UTF-8") as f:
        writer = csv.writer(f, delimiter='\t')
        writer.writerow(new_head)
        for line in lines:
            writer.writerow([line[0],line[1],line[2],synset2str(line[3]),synset2str(line[4])])

In [ ]:
write_prediction(wn_mrs_prediction_file, wn_predict_200)
accuracy1, accuracy2 = validation(wn_predict_200,wn_gold_200)
accuracy1, accuracy2

## Frequent Baseline

In [ ]:
def MFS(wn_gold):
    # Frequent Baseline Calculation
    wn_predict = []
    for line in wn_gold:
        label1  = line[0]
        label2 = line[2]
        relationship = line[1]
        
        candidates1 = generate_candidates(label1)
        candidates2 = generate_candidates(label2)
        
        if candidates1:
            node1_id = candidates1[0]
        else:
            #print(label1)
            #_ = label1
            node1_id = ""
        
        if candidates2:
            node2_id = candidates2[0]
        else:
            #print(label2)
            #_ = label2
            node2_id = ""
        
        #print(node2_id)
        wn_predict.append([label1, relationship, label2, node1_id, node2_id])
        
    return wn_predict

In [ ]:
wn_predict_200 = MFS(wn_gold_200)
wn_predict_200[:10]

In [ ]:
write_prediction(wn_mfs_prediction_file, wn_predict_200)
accuracy1, accuracy2 = validation(wn_predict_200,wn_gold_200)
accuracy1, accuracy2

## sentence-transformer-bert

In [ ]:
model_STB = SentenceTransformer('roberta-large-nli-stsb-mean-tokens')

In [ ]:
# check the relationship
relationships = set()

for line in wn_gold_all:
    relationships.add(line[1])

In [ ]:
relationships

In [ ]:
word2sentence = {'/r/IsA':"is a", '/r/MadeOf': "is made of",'/r/PartOf':"is part of"}

In [ ]:
def line_sentence(line, word2sentence):
    sentence = line[0]+" "+word2sentence[line[1]]+" "+line[2]
    return sentence

def label2sentence2embed(label,embeddings, model):
    temp = []
    if label in embeddings:
        pass
    else:
        candidates = generate_candidates(label)

        for candit in candidates:
            temp.append([candit,model.encode(candit.definition())])
            
        embeddings[label] = temp
            
    return embeddings

def candidates_embeddings(wn_gold, model):
    embeddings = dict()
    i = 0
    for line in wn_gold:
        label1  = line[0]
        label2 = line[2]
        print("\r",i, end="")
        i +=1
        embeddings = label2sentence2embed(label1,embeddings, model)
        embeddings = label2sentence2embed(label2,embeddings, model)
    return embeddings


def STB(wn_gold, model = None, label_embeddings = None, word2sentence = None):
    # sentence-transformer-bert Calculation
    wn_predict = []
    i = 0
    for line in wn_gold:
        label1 = line[0]
        label2 = line[2]
        sentence=line_sentence(line, word2sentence)
        sent_embedding = model.encode(sentence)

        #obtain the max similar item for label1
        max_item = [0,""]
        if label1 not in label_embeddings:
            node_id1 = ""
        else:
            for candit, embedding in label_embeddings[label1]:
                similar = rltk.cosine_similarity(list(embedding), list(sent_embedding))
                temp = [similar, candit]

                if temp>max_item:
                    max_item = temp

            node_id1 = max_item[-1]
        
        #obtain the max similar item for label2
        max_item = [0,""]
        if label2 not in label_embeddings:
            node_id2 = ""
        else:
            for candit, embedding in label_embeddings[label2]:
                similar = rltk.cosine_similarity(list(embedding), list(sent_embedding))
                temp = [similar, candit]

                if temp>max_item:
                    max_item = temp

            node_id2 = max_item[-1]
        print("\r",i, end="")
        i +=1
                
        wn_predict.append([label1, line[1], label2,node_id1,node_id2])
        
    return wn_predict

In [ ]:
label_embeddings = candidates_embeddings(wn_gold_200, model_STB)

In [ ]:
wn_predict_200 = STB(wn_gold_200, model = model_STB, label_embeddings = label_embeddings, word2sentence = word2sentence)

In [ ]:
write_prediction(wn_stb_prediction_file, wn_predict_200)
accuracy1, accuracy2 = validation(wn_predict_200,wn_gold_200)
accuracy1, accuracy2

## sentence-transformer-roberta

In [ ]:
model_STR = SentenceTransformer('bert-large-nli-stsb-mean-tokens')

In [ ]:
def STR(wn_gold, model = None, label_embeddings = label_embeddings, word2sentence = word2sentence):
    return STB(wn_gold, model = model, label_embeddings = label_embeddings, word2sentence = word2sentence)

In [ ]:
label_embeddings = candidates_embeddings(wn_gold_200, model_STR)

In [ ]:
wn_predict_200 = STR(wn_gold_200, model = model_STR, label_embeddings = label_embeddings, word2sentence = word2sentence)

In [ ]:
write_prediction(wn_str_prediction_file, wn_predict_200)
accuracy1, accuracy2 = validation(wn_predict_200,wn_gold_200)
accuracy1, accuracy2

## Compare

In [ ]:
accuracy_MRS = []
accuracy_MFS = []
accuracy_STB = []
accuracy_STR = []
n=10
for i in range(n):
    print(i)
    wn_gold_200 = random.choices(wn_gold_all, k=200)
    wn_predict_200 = MRS(wn_gold_200)
    accuracy1, accuracy2 = validation(wn_predict_200,wn_gold_200)
    accuracy_MRS.append(accuracy1)
    
    wn_predict_200 = MFS(wn_gold_200)
    accuracy1, accuracy2 = validation(wn_predict_200,wn_gold_200)
    accuracy_MFS.append(accuracy1)
    
    label_embeddings = candidates_embeddings(wn_gold_200, model_STB)
    wn_predict_200 = STB(wn_gold_200, model = model_STB, label_embeddings = label_embeddings, word2sentence = word2sentence)
    accuracy1, accuracy2 = validation(wn_predict_200,wn_gold_200)
    accuracy_STB.append(accuracy1)
    
    label_embeddings = candidates_embeddings(wn_gold_200, model_STR)
    wn_predict_200 = STR(wn_gold_200, model = model_STR, label_embeddings = label_embeddings, word2sentence = word2sentence)
    accuracy1, accuracy2 = validation(wn_predict_200,wn_gold_200)
    accuracy_STR.append(accuracy1)
    

In [ ]:
n=10
x_axis = range(n)
plt.plot(x_axis, accuracy_MRS, color='green', label='MRS')
plt.plot(x_axis, accuracy_MFS, color='red', label='MFS')
plt.plot(x_axis, accuracy_STB,  color='skyblue', label='STB')
plt.plot(x_axis, accuracy_STR, color='blue', label='STR')
plt.legend()

plt.xlabel('iteration times')
plt.ylabel('accuracy')
plt.show()

## WordNet graph

In [ ]:
head, lines = load_file(kgtk_cn_file)
lines_1k = random.choices(lines, k=1000)

In [ ]:
cn_test_all = generate_gold_file(lines)

In [ ]:
cn_test_1k = generate_gold_file(lines_1k)

In [ ]:
new_head = ['node1;label','relation','node2;label','node1','node2']
# write in to file 200 records
with open(cn_test_1k_file, "w", newline='', encoding="utf-8") as f:
    writer = csv.writer(f, delimiter='\t')
    writer.writerow(new_head)
    writer.writerows(cn_test_1k)

In [ ]:
cn_predict_1k = MFS(cn_test_1k)
write_prediction(cn_prediction_file, cn_predict_1k)
cn_predict_1k[0]

# Frequency Mehotd

In [ ]:
def no_synset_prob(cn_predict_1k):
    #prob 1: no synset for label
    #prob 2: no synset for record
    count1 = 0
    count2 = 0
    for record in cn_predict_1k:
        judge = [synset2str(record[3]) == "wn:",synset2str(record[4]) == "wn:"]
        
        if judge[0]:
            count1 += 1
            
        if judge[1]:
            count1 += 1
            
        if any(judge):
            count2 += 1
            
    return count1/(2*len(cn_predict_1k)), count2/len(cn_predict_1k)

In [ ]:
prob1,prob2 = no_synset_prob(cn_predict_1k)
prob1, prob2

## sentence-transformer-roberta for WordNet

In [ ]:
label_embeddings = candidates_embeddings(cn_test_1k, model_STR)

In [ ]:
relationships = set()

for line in lines:
    relationships.add(line[1])
    
relationships

In [ ]:
word2sentence = {'/r/Antonym':"is antonym for", 
                 '/r/AtLocation': "is located at",
                 '/r/CapableOf':"is capable of",
                '/r/Causes':"causes",
                '/r/CausesDesire':"causes the desire of",
                '/r/CreatedBy':"is created by",
                '/r/DefinedAs': " is defined as",
                '/r/DerivedFrom': "is derived from",
                '/r/Desires':"desires",
                '/r/DistinctFrom':"is distinct from",
                "/r/Entails":"entails",
                '/r/EtymologicallyDerivedFrom':"is etymologically derived from",
                '/r/EtymologicallyRelatedTo': "is etymologically related to",
                '/r/FormOf':"is form of",
                '/r/HasA': "has a",
                '/r/HasContext': "has the context of",
                '/r/HasFirstSubevent': "has first subevent, ",
                '/r/HasLastSubevent':"has last subevent, ",
                '/r/HasPrerequisite': "has prerequisite, ",
                '/r/HasProperty': "has property, ",
                '/r/HasSubevent': "has subevent, ",
                '/r/InstanceOf': " is an instance of",
                '/r/IsA': "is a",
                '/r/LocatedNear': "is located nearby",
                '/r/MadeOf': "is made of",
                '/r/MannerOf':"has a manner of",
                '/r/MotivatedByGoal': "is motivated by goal",
                '/r/NotCapableOf': "is not capable of",
                '/r/NotDesires':"does not desire",
                '/r/NotHasProperty':"does not have property, ",
                '/r/PartOf': "is part of",
                '/r/ReceivesAction':"receives the action, ",
                '/r/RelatedTo':"is related to",
                '/r/SimilarTo':"is similar to",
                '/r/SymbolOf':"is a symbol of",
                '/r/Synonym':"is synonym for",
                '/r/UsedFor':"is used for",
                '/r/dbpedia/capital': "is the capital of",
                '/r/dbpedia/field':" is the field of",
                '/r/dbpedia/genre':"has genre,",
                '/r/dbpedia/genus':"has genus, ",
                '/r/dbpedia/influencedBy':"is influenced by",
                '/r/dbpedia/knownFor': "is known for",
                '/r/dbpedia/language':"is the language ",
                '/r/dbpedia/leader':"has the leader, ",
                '/r/dbpedia/occupation':"has the occupation, ",
                '/r/dbpedia/product':"has the product, "}

cn_predict_1k = STR(cn_test_1k, model = model_STR, label_embeddings = label_embeddings, word2sentence = word2sentence)
write_prediction(cn_prediction_file, cn_predict_1k)
prob1,prob2 = no_synset_prob(cn_predict_1k)
prob1, prob2

In [ ]:
distri = distribution(cn_test_1k)
plt.hist(distri.values(),log=True,bins=30)
print("mean ambiguity of label:", sum(distri.values())/len(distri), "size of records:", len(cn_test_1k),"num of distinct labels:", len(distri))